# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [43]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [44]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [45]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    
    sourceSentences = [sentence for sentence in source_text.split('\n')]
    # remember to add <EOS> to target sentence as per instructions
    targetSentences = [sentence + ' <EOS>' for sentence in target_text.split('\n')]
    
    sourceIdText = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in sourceSentences]
    targetIdText = [[target_vocab_to_int[word] for word in sentence.split()] for sentence in targetSentences]
    
    #print( sourceIdText )
    #print( targetIdText )
    return sourceIdText, targetIdText

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [46]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [47]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [48]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [49]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    # the sequence_to_sequence_implementation in dl class shows how to do this
    #
    input = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learningRate = tf.placeholder(tf.float32, name='learning_rate', shape=())
    
    # for unit test keepProbability unit test asserts and expects 
    # keep_prob , as per comments
    keepProbability = tf.placeholder(tf.float32, name='keep_prob',  shape=())

    targetSequenceLength = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    maxTargetSequenceLength = tf.reduce_max(targetSequenceLength, name='max_target_sequence_length')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')

    return input, targets, learningRate, keepProbability, targetSequenceLength, maxTargetSequenceLength, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [50]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    # once again we can take what we learned from the 
    # the sequence to sequence lesson
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    decInput = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return decInput

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [51]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    # fortunately in my previous work I figured out how to add
    # keep probability for the drop out layer
    # we can use what we learned with seq2seq and expand it
    # Encoder embedding
    encEmbedInput = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)

    # RNN cell
    def make_cell(rnn_size):
        encCell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        drop = tf.contrib.rnn.DropoutWrapper(encCell, output_keep_prob=keep_prob)
        return drop

    encCell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    encOutput, encState = tf.nn.dynamic_rnn(encCell, encEmbedInput, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return encOutput, encState

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [52]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    # Helper for the training process. Used by BasicDecoder to read inputs.
    # once 
    # apply dropout to our input layer
    # dropoutAppliedLayer = tf.layers.dropout( dec_embed_input, keep_prob )
    
    trainingHelper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=target_sequence_length,
                                                        time_major=False)
    # Basic decoder
    trainingDecoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       trainingHelper,
                                                       encoder_state,
                                                       output_layer) 
    
    
    
    # Perform dynamic decoding using the decoder
    trainingDecoderOutput = tf.contrib.seq2seq.dynamic_decode(trainingDecoder,
                                                                impute_finished=True,
                                                                maximum_iterations=max_summary_length)[0]

    return trainingDecoderOutput



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [53]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding  ? where is this???
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    # similiar to other seq2seq but need layer and sequence ids
    #
    startTokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')

    # Helper for the inference process.
    inferenceHelper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                            startTokens,
                                                            end_of_sequence_id)

    # Basic decoder
    inferenceDecoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                    inferenceHelper,
                                                    encoder_state,
                                                    output_layer)

    # Perform dynamic decoding using the decoder
    inferenceDecoderOutput = tf.contrib.seq2seq.dynamic_decode(inferenceDecoder,
                                                        impute_finished=True,
                                                        maximum_iterations=max_target_sequence_length)[0]

    return inferenceDecoderOutput


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [54]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # seq2seq moodified
    
    outputLayer = Dense(target_vocab_size, 
                         kernel_initializer=tf.truncated_normal_initializer(stddev=0.1))
    
    decoderEmbeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size], minval=-1, maxval=1))
    
    decoderEmbeddedInput = tf.nn.embedding_lookup(decoderEmbeddings, dec_input)
    
    def buildLSTMCell(rnn_size):
        
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size)
        
        return tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
    
    decoderCells = tf.contrib.rnn.MultiRNNCell ( [buildLSTMCell(rnn_size) for _ in range(num_layers)])

    
    with tf.variable_scope('decoder'):
        
        trainDecoderLogits = decoding_layer_train( encoder_state, decoderCells, 
                                                   decoderEmbeddedInput, 
                                                   target_sequence_length,
                                                   max_target_sequence_length, 
                                                   outputLayer, 
                                                   keep_prob)

    with tf.variable_scope('decoder', reuse=True):
        
        infoDecorderLogits = decoding_layer_infer(  encoder_state, decoderCells, 
                                                    decoderEmbeddings, 
                                                    target_vocab_to_int['<GO>'], 
                                                    target_vocab_to_int['<EOS>'], 
                                                    max_target_sequence_length, 
                                                    target_vocab_size, 
                                                    outputLayer, 
                                                    batch_size, 
                                                    keep_prob)

    return trainDecoderLogits, infoDecorderLogits



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [56]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # seq2seq lesson saves the day again

    encOutput, encState = encoding_layer( input_data, rnn_size, num_layers, keep_prob, 
                                   source_sequence_length, source_vocab_size, enc_embedding_size)
    
    decoderInput = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
#     def decoding_layer(dec_input, encoder_state,
#                    target_sequence_length, max_target_sequence_length,
#                    rnn_size,
#                    num_layers, target_vocab_to_int, target_vocab_size,
#                    batch_size, keep_prob, decoding_embedding_size):
    
    return decoding_layer(decoderInput, encState,
                   target_sequence_length, max_target_sentence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, dec_embedding_size)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [57]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 100
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 64
decoding_embedding_size = 64
# Learning Rate
learning_rate = .01
# Dropout Keep Probability
keep_probability = 1
display_step = 5

### Build the Graph
Build the graph using the neural network you implemented.

In [58]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [61]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [60]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    5/1378 - Train Accuracy: 0.2640, Validation Accuracy: 0.3336, Loss: 3.9646
Epoch   0 Batch   10/1378 - Train Accuracy: 0.2970, Validation Accuracy: 0.3741, Loss: 3.1764
Epoch   0 Batch   15/1378 - Train Accuracy: 0.3281, Validation Accuracy: 0.3868, Loss: 2.7318
Epoch   0 Batch   20/1378 - Train Accuracy: 0.2611, Validation Accuracy: 0.3482, Loss: 2.4661
Epoch   0 Batch   25/1378 - Train Accuracy: 0.2821, Validation Accuracy: 0.3818, Loss: 2.0058
Epoch   0 Batch   30/1378 - Train Accuracy: 0.2775, Validation Accuracy: 0.3518, Loss: 1.6507
Epoch   0 Batch   35/1378 - Train Accuracy: 0.3871, Validation Accuracy: 0.4273, Loss: 1.2180
Epoch   0 Batch   40/1378 - Train Accuracy: 0.2805, Validation Accuracy: 0.3664, Loss: 1.2466
Epoch   0 Batch   45/1378 - Train Accuracy: 0.3725, Validation Accuracy: 0.4436, Loss: 1.0932
Epoch   0 Batch   50/1378 - Train Accuracy: 0.3530, Validation Accuracy: 0.4391, Loss: 1.0743
Epoch   0 Batch   55/1378 - Train Accuracy: 0.3158, Validati

Epoch   0 Batch  445/1378 - Train Accuracy: 0.7362, Validation Accuracy: 0.6791, Loss: 0.3034
Epoch   0 Batch  450/1378 - Train Accuracy: 0.7085, Validation Accuracy: 0.7291, Loss: 0.3399
Epoch   0 Batch  455/1378 - Train Accuracy: 0.7319, Validation Accuracy: 0.7268, Loss: 0.3287
Epoch   0 Batch  460/1378 - Train Accuracy: 0.6963, Validation Accuracy: 0.7232, Loss: 0.3735
Epoch   0 Batch  465/1378 - Train Accuracy: 0.6970, Validation Accuracy: 0.6973, Loss: 0.3391
Epoch   0 Batch  470/1378 - Train Accuracy: 0.7419, Validation Accuracy: 0.7227, Loss: 0.2884
Epoch   0 Batch  475/1378 - Train Accuracy: 0.6935, Validation Accuracy: 0.7168, Loss: 0.3116
Epoch   0 Batch  480/1378 - Train Accuracy: 0.7490, Validation Accuracy: 0.7491, Loss: 0.3263
Epoch   0 Batch  485/1378 - Train Accuracy: 0.7016, Validation Accuracy: 0.7241, Loss: 0.3676
Epoch   0 Batch  490/1378 - Train Accuracy: 0.6629, Validation Accuracy: 0.7123, Loss: 0.3153
Epoch   0 Batch  495/1378 - Train Accuracy: 0.7221, Validati

Epoch   0 Batch  885/1378 - Train Accuracy: 0.8047, Validation Accuracy: 0.8182, Loss: 0.2117
Epoch   0 Batch  890/1378 - Train Accuracy: 0.8745, Validation Accuracy: 0.8155, Loss: 0.1553
Epoch   0 Batch  895/1378 - Train Accuracy: 0.8332, Validation Accuracy: 0.7632, Loss: 0.1658
Epoch   0 Batch  900/1378 - Train Accuracy: 0.8355, Validation Accuracy: 0.8027, Loss: 0.1674
Epoch   0 Batch  905/1378 - Train Accuracy: 0.8120, Validation Accuracy: 0.8027, Loss: 0.1623
Epoch   0 Batch  910/1378 - Train Accuracy: 0.8495, Validation Accuracy: 0.7782, Loss: 0.1530
Epoch   0 Batch  915/1378 - Train Accuracy: 0.8567, Validation Accuracy: 0.8095, Loss: 0.1486
Epoch   0 Batch  920/1378 - Train Accuracy: 0.8419, Validation Accuracy: 0.8336, Loss: 0.1444
Epoch   0 Batch  925/1378 - Train Accuracy: 0.8510, Validation Accuracy: 0.8114, Loss: 0.1673
Epoch   0 Batch  930/1378 - Train Accuracy: 0.8610, Validation Accuracy: 0.8123, Loss: 0.1358
Epoch   0 Batch  935/1378 - Train Accuracy: 0.8605, Validati

Epoch   0 Batch 1325/1378 - Train Accuracy: 0.9533, Validation Accuracy: 0.8827, Loss: 0.0598
Epoch   0 Batch 1330/1378 - Train Accuracy: 0.9132, Validation Accuracy: 0.9059, Loss: 0.0879
Epoch   0 Batch 1335/1378 - Train Accuracy: 0.9305, Validation Accuracy: 0.9050, Loss: 0.0760
Epoch   0 Batch 1340/1378 - Train Accuracy: 0.9120, Validation Accuracy: 0.9082, Loss: 0.0726
Epoch   0 Batch 1345/1378 - Train Accuracy: 0.9243, Validation Accuracy: 0.9077, Loss: 0.0606
Epoch   0 Batch 1350/1378 - Train Accuracy: 0.9280, Validation Accuracy: 0.9323, Loss: 0.0682
Epoch   0 Batch 1355/1378 - Train Accuracy: 0.9147, Validation Accuracy: 0.9209, Loss: 0.0780
Epoch   0 Batch 1360/1378 - Train Accuracy: 0.8970, Validation Accuracy: 0.9059, Loss: 0.0913
Epoch   0 Batch 1365/1378 - Train Accuracy: 0.9220, Validation Accuracy: 0.9050, Loss: 0.0650
Epoch   0 Batch 1370/1378 - Train Accuracy: 0.9415, Validation Accuracy: 0.9232, Loss: 0.0611
Epoch   0 Batch 1375/1378 - Train Accuracy: 0.9448, Validati

Epoch   1 Batch  390/1378 - Train Accuracy: 0.9729, Validation Accuracy: 0.9300, Loss: 0.0402
Epoch   1 Batch  395/1378 - Train Accuracy: 0.9065, Validation Accuracy: 0.9218, Loss: 0.0734
Epoch   1 Batch  400/1378 - Train Accuracy: 0.9435, Validation Accuracy: 0.9223, Loss: 0.0580
Epoch   1 Batch  405/1378 - Train Accuracy: 0.9586, Validation Accuracy: 0.9255, Loss: 0.0371
Epoch   1 Batch  410/1378 - Train Accuracy: 0.9505, Validation Accuracy: 0.9168, Loss: 0.0423
Epoch   1 Batch  415/1378 - Train Accuracy: 0.9390, Validation Accuracy: 0.9145, Loss: 0.0297
Epoch   1 Batch  420/1378 - Train Accuracy: 0.9375, Validation Accuracy: 0.9050, Loss: 0.0674
Epoch   1 Batch  425/1378 - Train Accuracy: 0.9610, Validation Accuracy: 0.9127, Loss: 0.0284
Epoch   1 Batch  430/1378 - Train Accuracy: 0.9505, Validation Accuracy: 0.9086, Loss: 0.0667
Epoch   1 Batch  435/1378 - Train Accuracy: 0.9600, Validation Accuracy: 0.9400, Loss: 0.0375
Epoch   1 Batch  440/1378 - Train Accuracy: 0.9270, Validati

...

Epoch   8 Batch  885/1378 - Train Accuracy: 0.9579, Validation Accuracy: 0.9555, Loss: 0.0561
Epoch   8 Batch  890/1378 - Train Accuracy: 0.9470, Validation Accuracy: 0.9214, Loss: 0.0394
Epoch   8 Batch  895/1378 - Train Accuracy: 0.9558, Validation Accuracy: 0.9327, Loss: 0.0298
Epoch   8 Batch  900/1378 - Train Accuracy: 0.9825, Validation Accuracy: 0.9541, Loss: 0.0230
Epoch   8 Batch  905/1378 - Train Accuracy: 0.9670, Validation Accuracy: 0.9486, Loss: 0.0322
Epoch   8 Batch  910/1378 - Train Accuracy: 0.9675, Validation Accuracy: 0.9459, Loss: 0.0315
Epoch   8 Batch  915/1378 - Train Accuracy: 0.9405, Validation Accuracy: 0.9455, Loss: 0.0508
Epoch   8 Batch  920/1378 - Train Accuracy: 0.9729, Validation Accuracy: 0.9368, Loss: 0.0265
Epoch   8 Batch  925/1378 - Train Accuracy: 0.9720, Validation Accuracy: 0.9468, Loss: 0.0203
Epoch   8 Batch  930/1378 - Train Accuracy: 0.9730, Validation Accuracy: 0.9505, Loss: 0.0345
Epoch   8 Batch  935/1378 - Train Accuracy: 0.9625, Validati

Epoch   8 Batch 1325/1378 - Train Accuracy: 0.9662, Validation Accuracy: 0.9527, Loss: 0.0168
Epoch   8 Batch 1330/1378 - Train Accuracy: 0.9600, Validation Accuracy: 0.9505, Loss: 0.0443
Epoch   8 Batch 1335/1378 - Train Accuracy: 0.9847, Validation Accuracy: 0.9400, Loss: 0.0168
Epoch   8 Batch 1340/1378 - Train Accuracy: 0.9555, Validation Accuracy: 0.9368, Loss: 0.0339
Epoch   8 Batch 1345/1378 - Train Accuracy: 0.9757, Validation Accuracy: 0.9382, Loss: 0.0249
Epoch   8 Batch 1350/1378 - Train Accuracy: 0.9570, Validation Accuracy: 0.9414, Loss: 0.0374
Epoch   8 Batch 1355/1378 - Train Accuracy: 0.9516, Validation Accuracy: 0.9473, Loss: 0.0423
Epoch   8 Batch 1360/1378 - Train Accuracy: 0.9785, Validation Accuracy: 0.9459, Loss: 0.0303
Epoch   8 Batch 1365/1378 - Train Accuracy: 0.9730, Validation Accuracy: 0.9482, Loss: 0.0312
Epoch   8 Batch 1370/1378 - Train Accuracy: 0.9605, Validation Accuracy: 0.9623, Loss: 0.0405
Epoch   8 Batch 1375/1378 - Train Accuracy: 0.9876, Validati

Epoch   9 Batch  390/1378 - Train Accuracy: 0.9767, Validation Accuracy: 0.9209, Loss: 0.0268
Epoch   9 Batch  395/1378 - Train Accuracy: 0.9525, Validation Accuracy: 0.9432, Loss: 0.0450
Epoch   9 Batch  400/1378 - Train Accuracy: 0.9640, Validation Accuracy: 0.9186, Loss: 0.0513
Epoch   9 Batch  405/1378 - Train Accuracy: 0.9719, Validation Accuracy: 0.9145, Loss: 0.0359
Epoch   9 Batch  410/1378 - Train Accuracy: 0.9705, Validation Accuracy: 0.9141, Loss: 0.0323
Epoch   9 Batch  415/1378 - Train Accuracy: 0.9500, Validation Accuracy: 0.9300, Loss: 0.0243
Epoch   9 Batch  420/1378 - Train Accuracy: 0.9740, Validation Accuracy: 0.9305, Loss: 0.0409
Epoch   9 Batch  425/1378 - Train Accuracy: 0.9575, Validation Accuracy: 0.9300, Loss: 0.0250
Epoch   9 Batch  430/1378 - Train Accuracy: 0.9467, Validation Accuracy: 0.9282, Loss: 0.0399
Epoch   9 Batch  435/1378 - Train Accuracy: 0.9732, Validation Accuracy: 0.9505, Loss: 0.0330
Epoch   9 Batch  440/1378 - Train Accuracy: 0.9635, Validati

Epoch   9 Batch  830/1378 - Train Accuracy: 0.9548, Validation Accuracy: 0.9495, Loss: 0.0299
Epoch   9 Batch  835/1378 - Train Accuracy: 0.9805, Validation Accuracy: 0.9414, Loss: 0.0322
Epoch   9 Batch  840/1378 - Train Accuracy: 0.9730, Validation Accuracy: 0.9555, Loss: 0.0337
Epoch   9 Batch  845/1378 - Train Accuracy: 0.9780, Validation Accuracy: 0.9595, Loss: 0.0199
Epoch   9 Batch  850/1378 - Train Accuracy: 0.9760, Validation Accuracy: 0.9518, Loss: 0.0249
Epoch   9 Batch  855/1378 - Train Accuracy: 0.9657, Validation Accuracy: 0.9709, Loss: 0.0196
Epoch   9 Batch  860/1378 - Train Accuracy: 0.9626, Validation Accuracy: 0.9550, Loss: 0.0457
Epoch   9 Batch  865/1378 - Train Accuracy: 0.9665, Validation Accuracy: 0.9514, Loss: 0.0335
Epoch   9 Batch  870/1378 - Train Accuracy: 0.9763, Validation Accuracy: 0.9550, Loss: 0.0202
Epoch   9 Batch  875/1378 - Train Accuracy: 0.9620, Validation Accuracy: 0.9714, Loss: 0.0243
Epoch   9 Batch  880/1378 - Train Accuracy: 0.9660, Validati

Epoch   9 Batch 1270/1378 - Train Accuracy: 0.9850, Validation Accuracy: 0.9623, Loss: 0.0318
Epoch   9 Batch 1275/1378 - Train Accuracy: 0.9660, Validation Accuracy: 0.9514, Loss: 0.0262
Epoch   9 Batch 1280/1378 - Train Accuracy: 0.9624, Validation Accuracy: 0.9432, Loss: 0.0342
Epoch   9 Batch 1285/1378 - Train Accuracy: 0.9750, Validation Accuracy: 0.9364, Loss: 0.0202
Epoch   9 Batch 1290/1378 - Train Accuracy: 0.9775, Validation Accuracy: 0.9500, Loss: 0.0301
Epoch   9 Batch 1295/1378 - Train Accuracy: 0.9870, Validation Accuracy: 0.9277, Loss: 0.0257
Epoch   9 Batch 1300/1378 - Train Accuracy: 0.9710, Validation Accuracy: 0.9518, Loss: 0.0497
Epoch   9 Batch 1305/1378 - Train Accuracy: 0.9663, Validation Accuracy: 0.9514, Loss: 0.0480
Epoch   9 Batch 1310/1378 - Train Accuracy: 0.9405, Validation Accuracy: 0.9527, Loss: 0.0438
Epoch   9 Batch 1315/1378 - Train Accuracy: 0.9530, Validation Accuracy: 0.9518, Loss: 0.0296
Epoch   9 Batch 1320/1378 - Train Accuracy: 0.9658, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [62]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [63]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [64]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    # convert to lower case to reduce number of words
    sentence = sentence.lower();
    # split sentence into words
    words = sentence.split(); 
    # create a default for list get so we don't throw an error
    defaultNotFound = vocab_to_int['<UNK>']
    return  [vocab_to_int.get(word, defaultNotFound ) for word in words]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [65]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [147, 148, 159, 95, 117, 170, 68]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [248, 176, 273, 53, 31, 147, 288, 39, 1]
  French Words: il a vu une vieille voiture blanche . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.